In [0]:
%run "../../../config/setup"

In [0]:
CREATE OR REPLACE VIEW preferences_customer_dedup AS
  SELECT
    customer_id,
    max(pref_value) AS pref_value,
    max(preference_category) AS preference_category,
    max(source_file) AS source_file,
    max(ingest_ts) AS ingest_ts
  FROM preferences_bronze
  WHERE customer_id IS NOT NULL
  GROUP BY customer_id;

In [0]:
GRANT SELECT ON TABLE preferences_customer_dedup TO devs;

In [0]:
-- SELECT max(row_count) >= 1 FROM
--   (SELECT
--     customer_id,
--     count(*) AS row_count
--   FROM preferences_customer_dedup
--   WHERE customer_id IS NOT NULL
--   GROUP BY customer_id);

In [0]:
CREATE OR REPLACE VIEW preferences_silver AS
  SELECT
    customer_id,
    pref_value,
    CASE
      WHEN LENGTH(preference_category) > 0 THEN preference_category 
      ELSE NULL 
    END AS preference_category,
    CAST(ingest_ts AS DATE) AS date
  FROM preferences_customer_dedup
  WHERE pref_value IS NOT NULL

In [0]:
GRANT SELECT ON TABLE preferences_silver TO devs;